In [58]:
import csv
import numpy as np
import torch
from torch import optim
import random 
from pytorch_transformers.tokenization_distilbert import DistilBertTokenizer
from pytorch_transformers.modeling_distilbert import DistilBertModel

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
special_tokens_dict = {'additional_special_tokens': ['<PLH>', '<s>','[END]']}
tokenizer.add_special_tokens(special_tokens_dict)
encoder = DistilBertModel.from_pretrained('distilbert-base-uncased').cuda()
encoder.resize_token_embeddings(len(tokenizer))

for param in encoder.parameters():
    param.requires_grad = False

pad_token = tokenizer.pad_token
train_data = []
test_data = []
    
with open("memory_conversation.tsv") as csvDataFile:
    csvreader = csv.reader(csvDataFile, delimiter='\t')
    next(csvreader)
    
    data = []
    conversation = {
                "conv_id":None,
                "exchange_ids":[],
                "user_responses":[],
                "num_states":1,
                "state_updates":[],
                "agent_utterances":[],
                "state_embeddings":[]
            }
    max_states = 0
    max_utterances = 0
    
    for conv_id, exchange_id, agent, user, action1, action2, action3 in csvreader:
        if conversation["conv_id"] is None:
            conversation["conv_id"] = conv_id
        elif conv_id != conversation["conv_id"]:
            data.append(conversation)
            max_utterances = max(
                max_utterances, 
                len(conversation["exchange_ids"])
            )
            conversation = {
                "conv_id":conv_id,
                "exchange_ids":[],
                "user_responses":[],
                "num_states":1,
                "state_updates":[],
                "agent_utterances":[],
                "state_embeddings":[]
            }
        
        conversation["agent_utterances"].append(agent)
        conversation["user_responses"].append(user)
        
        state_updates = []
        for action in (action1, action2, action3):
            if action != "NULL":
                tokenized_action = torch.cuda.LongTensor([
                    tokenizer.encode(
                        action.split(", ")[1]
                    )
                ])
                embedding = encoder(tokenized_action)[0]
                idx = conversation["num_states"]
                conversation["num_states"] += 1
                state_updates.append((idx, embedding))
        conversation["state_updates"] = state_updates
        conversation["exchange_ids"].append(exchange_id)
        max_states = max(max_states, len(conversation["exchange_ids"]))
    if len(conversation) > 0:
        data.append(conversation)
    max_utterances = max(
        max_utterances, 
        len(conversation["agent_utterances"])
    )

In [59]:
class Decoder(torch.nn.Module):
    def __init__(self, vsz, esz=768, hsz=512, num_layers=4, num_heads=8):
        super().__init__()
        self.attn = []
        self.dense = []
        self.relu = []
        self.num_layers = num_layers
        self.norm = []
        
        for i in range(num_layers):
            if i == 0:
                self.attn.append(
                    torch.nn.MultiheadAttention(esz, num_heads, dropout=0.1).cuda()
                )
                self.dense.append(
                    torch.nn.Linear(
                        esz, hsz,
                    ).cuda()
                )
            else:
                self.attn.append(
                    torch.nn.MultiheadAttention(hsz, num_heads, dropout=0.1).cuda()
                )
                self.dense.append(
                    torch.nn.Linear(
                        hsz, hsz,
                    ).cuda()
                )
            self.relu.append(torch.nn.ReLU().cuda())
            self.norm.append(torch.nn.LayerNorm((300,1,hsz)).cuda())
        self.out = torch.nn.Linear(
            hsz, vsz,
        ).cuda()
        
    # accepts seq_len x num_values  returns num_keys
    def forward(self, input, hidden=None):
        input, mask = input
        for i in range(self.num_layers):
            attn_mask, _ = self.attn[i](input, input, input, key_padding_mask=mask)
            input_a = attn_mask + input
            input_b = self.relu[i](self.dense[i](input_a))
            input = self.norm[i](input_b)
        
        return self.out(input)

In [60]:
class Model():
    def __init__(self, lr=0.0005, hsz=768, pad_len=300):
        self.pad_len = pad_len
        self.decoder = Decoder(len(tokenizer)).cuda()
        self.optims = {
            'decoder': optim.Adam(self.decoder.parameters(), lr=lr),
        }
        self.loss = 0
        self.step = 0
        loss_weights = torch.ones(len(tokenizer))
        #loss_weights[tokenizer.encode("[END]")[0]] = 2
        self.criterion = torch.nn.CrossEntropyLoss(ignore_index=0)
    
    def zero_grad(self):
        for optimizer in self.optims.values():
            optimizer.zero_grad()
            
    def update_params(self):
        for optimizer in self.optims.values():
            optimizer.step()
    
    def encode_option(self, i, conversation, dialog):
        prefix = conversation["exchange_ids"][i][:-1]
        ids, utterances, responses = [], [], []
        level = conversation["exchange_ids"][i].count("_")
        while conversation["exchange_ids"][i].count("_") == level:
            utterances.append(conversation["agent_utterances"][i])
            responses.append(conversation["user_responses"][i])
            ids.append(conversation["exchange_ids"][i])
            i += 1

        idx = random.choice(range(len(ids)))
        id = ids[idx]
        dialog.append((utterances[idx], responses[idx]))

        while prefix in conversation["exchange_ids"][i]:
            
            if id not in conversation["exchange_ids"][i]:
                i += 1
                continue

            if conversation["exchange_ids"][i].count("_") == level + 1:
                dialog.append(
                    (conversation["agent_utterances"][i], 
                    conversation["user_responses"][i])
                )
                i += 1
            elif conversation["exchange_ids"][i].count("_") == level + 2:
                i, _ = self.encode_option(i, conversation, dialog)
            else:
                raise Exception()
        return i, dialog
        
    def tokenize(self, text):
        tokens = tokenizer.encode(text)
        while len(tokens) < self.pad_len:
            tokens.append(tokenizer.pad_token_id)
        return torch.cuda.LongTensor([tokens])
        
    def encode(self, text):
        tokens = self.tokenize(text)
        return encoder(tokens)[0], tokens.long()
            
    def generate_turn(self, conversation, wsz=5):
        history = []
        i = 0
        while i < len(conversation["exchange_ids"]):
            if len(history) > wsz:
                history = history[len(history) - wsz:]
            exchange_id = conversation["exchange_ids"][i]
            if not exchange_id.endswith("_A"):
                agent = random.choice(conversation["agent_utterances"][i].split("|"))
                user = conversation["user_responses"][i]
                
                if user == "NULL":
                    if len(history) == 0:
                        history += ["[AGENT] " + agent]
                        continue
                    current_encoded, current_tokens = self.encode(" ".join(history +  [" [END]"]))
                    history += ["[AGENT] " + agent]
                    next_encoded, next_tokens = self.encode("[AGENT] " + agent +  " [END]")
                    yield current_encoded, current_tokens, next_tokens
                else:
                    history += ["[USER] " + user]
                i += 1
            else:
                i, dialog = self.encode_option(i, conversation, [])
                for agent, user in dialog:
                    if user == "NULL":
                        current_encoded, current_tokens = self.encode(" ".join(history + [" [END]"]))
                        history += ["[AGENT] " + random.choice(agent.split("|"))]
                        next_tokens = self.tokenize("[AGENT] " + random.choice(agent.split("|")) + " [END]")
                        yield current_encoded, current_tokens, next_tokens
                    else:
                        history += ["[USER] " + random.choice(user.split("|"))]
        
    def train_step(self):
        conversation = random.choice(data)
        self.zero_grad()
        loss = 0
        for current_encoded, current_tokens, prediction_tokens in self.generate_turn(conversation):
            mask = current_tokens == tokenizer.pad_token_id
            current_encoded = current_encoded.transpose(0,1)
            decoded = self.decoder((current_encoded, mask)).transpose(0,1).transpose(1,2)
            loss = self.criterion(decoded, prediction_tokens)
            self.step += 1
            self.loss += loss
            loss.backward()
            self.update_params()
            if self.step % 100 == 0:
                print("Step %d Loss %f" % (self.step, self.loss.item() / 50))
                self.loss = 0
                #self.eval_step()
                #decoded = 
                current_tokens = [t for t in current_tokens[0].tolist() if t != tokenizer.pad_token_id]
                print("Source")
                print(tokenizer.decode(current_tokens))
                print("Target")
                prediction_tokens = [t for t in prediction_tokens[0].tolist() if t != tokenizer.pad_token_id]
                print(tokenizer.decode(prediction_tokens))
                print("Predicted")
                #print(decoded.size())
                predicted_tokens = torch.argmax(decoded, 1)
                #print(predicted_tokens.size())
                predicted_tokens = [t for t  in predicted_tokens[0].tolist() if t != tokenizer.pad_token_id]
                #print(predicted_tokens)
                print(tokenizer.decode(predicted_tokens))
                #print(tokenizer.decode(predicted_tokens)[:500])
        
            
            

model = Model()
for i in range(1000):
    model.train_step()

Step 100 Loss 11.064176
Source
[ agent ] today, i want to talk about parts of the body. [ agent ] today, i want to talk about parts of the body. [END]
Target
[ agent ] first, let's talk about the face. [END]
Predicted
[ agent ] [END], let, [END]talk two [END]of [END] [END]] ] agent ] [END], let, there talk two [END] [END] [END] [END]] agent agent agent [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [
Step 200 Loss 6.745026
Source
[ agent ] ok, let's add numbers. [ agent ] what is one plus one? [ user ] two [ a

Step 1000 Loss 4.398754
Source
[ agent ] today we're going to learn numbers. [ agent ] today we're going to learn numbers. [ user ] ok [END]
Target
[ agent ] repeat after me. [END]
Predicted
[ agent ]'''''s [END] [END]?'agent ]'','' s seven. used'agent ] agent agent agent agent [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [
Step 1100 Loss 3.241523
Source
[ agent ] hello [ agent ] hello [ user ] hi! [END]
Target
[ agent ] how are you? [END]
Predicted
[ agent ] [END] [END]!. [END] [END] [END]. [END] [END] 

Step 2000 Loss 2.417148
Source
[ agent ] null [ agent ] null [ agent ] hi! [ agent ] did you want to talk about your family? [ user ] sure [END]
Target
[ agent ] let's start with your father. [END]
Predicted
[ agent ] with [END]father [END]with [END]father [END] [END]? [END]father? [END]' [END]? talk [END] [END]? [END] [END]agent s [END] [END] [END] [END]? [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [
Step 2100 Loss 2.427739
Source
[ agent ] null [END]
Target
[ agent ] null [END]
Predicted
[ agent ] null [E

Step 3000 Loss 1.721472
Source
[ user ] eleven [ agent ] twelve [ user ] twelve [ agent ] thirteen [ user ] thirteen [END]
Target
[ agent ] fourteen [END]
Predicted
[ agent ] fourteen [END]agent ] fifteen [END]agent ] fifteen [END]numbers ] sixteen [END] [END]] fifteen [END]] ] [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [
Step 3100 Loss 1.697524
Source
[ user ] four [ agent ] what is seventeen minus one? [ user ] sixteen [ agent ] ok, now ask me a question [ user ] what is twelve minus 

Step 3900 Loss 1.693449
Source
[ agent ] do you argue much? [ user ] sometimes! [ agent ] living together can be difficult. [ agent ] what does she do? [ user ] she is a student at university. [END]
Target
[ agent ] what is she studying? [END]
Predicted
[ agent ] what is she studying [END] [END]agent ] what [END] [END]agent ] will she be a that? [END]agent ] what is what what what [END]agent ] that's?? agent? [END] [END]] ] [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [
Step 4000 Loss 1.405495
Source
[ user ] eighteen [ agent ] w

Step 4800 Loss 1.060125
Source
[ agent ] have you seen a doctor? [ user ] not yet. [ user ] yes, i have. [ agent ] will you see a doctor? [ user ] yes, i will go tomorrow. [END]
Target
[ agent ] what did the doctor say? [END]
Predicted
[ agent ] [END]did the doctor say [END] [END] [END] [END]? [END]? [END]agent [END] [END] [END]what [END]? [END] [END]? what? [END]doctor say [END] [END] [END] [END]? [END]? what [END]?? [END] [END] [END]that [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [
Step 4900 Loss 1.258464
Source
[ agent ] good ev

Step 5800 Loss 1.257558
Source
[ user ] yes, i have. [ agent ] will you see a doctor? [ user ] yes, i will go tomorrow. [ agent ] what did the doctor say? [ user ] i need to rest for a few days. | i need to rest for a few weeks. | it will get better soon [END]
Target
[ agent ] ok, well i hope you feel better soon! [END]
Predicted
[ agent ] that old well you hope? agent better soon? what? say? you? better?? well soon s did hope? agent better is young is is is?? [END]better that you to to to you young you you [END]! you to to is! you you hope [END]you, you a that? [END]that that [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ 

Step 6600 Loss 1.351712
Source
[ agent ] is your father still alive? [ user ] no, he is dead. [ agent ] i'm sorry to hear that. [ agent ] what about your mother? [ user ] she is still alive. [END]
Target
[ agent ] that's good to hear. [END]
Predicted
[ agent ] that's good to hear? agent. how? [END]she this year [END]agent mother that'what a [END]young that?? agent? [END]that? did.... that she good [END]? [END] [END]] that [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [
Step 6700 Loss 1.118639
Source
[ agent ] did you want to talk about your

Step 7500 Loss 1.133973
Source
[ user ] there is a nose [ agent ] and? [ user ] ears [ agent ] no, ears are not on a face. [ user ] eyebrows [END]
Target
[ agent ] that's right, eyebrows are above the eyes. [END]
Predicted
[ agent ] that's right the eyebrows are above the eyes [END] [END] [END]eyes eyebrows are how the [END]? a [END] [END] [END]? the agent ] the the the that that [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [
Step 7600 Loss 0.716862
Source
[ agent ] how old are you? [ user ] i am 25. [ agent ] you

KeyboardInterrupt: 

In [54]:
[0,1,2,3,4,5,6,7][:-1]

[0, 1, 2, 3, 4, 5, 6]

In [ ]:
"foooo"[:5] [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ 

In [ ]:
data[2]